# <center> CIA World Factbook

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Load table

First we want to return information on the tables in the database.

In [4]:
%%sql 
SELECT * 
    FROM sqlite_master
  WHERE type = 'table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We will use facts table to analyze.

In [6]:
%%sql
SELECT * 
    FROM facts 
  LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

Descriptions for some ofthe columns is below.

- name : the name of the country.
- area : the country's total area (both land and water).
- area_land : the country's land area in square kilometers.
- area_water : the country's water area in square kilometers.
- population : the country's population.
- population_growth : the country's population growth as a percentage.
- birth_rate : the country's birth rate, or teh number of births per year per 1,000 people.
- death_rate : the country's death rate, or the number of death per year per 1,000 people. 

In [7]:
%%sql 
SELECT MIN(population) AS 'Minimum population', 
       MAX(population) AS 'Maximum population',
       MIN(population_growth) AS 'Minimum growth',
       MAX(population_growth) AS 'Maximum growth'
    FROM facts;

Done.


Minimum population,Maximum population,Minimum growth,Maximum growth
0,7256490011,0.0,4.02


There is some strange value in Minimum population and Minimum growth, we will check which country has that value.

### Check Outliers

1. Country with the minimum population

In [8]:
%%sql
SELECT name
    FROM facts
  WHERE population = (SELECT MIN(population) 
                          FROM facts);

Done.


name
Antarctica


2. Country with the maximum population

In [9]:
%%sql
SELECT name 
    FROM facts
  WHERE population = (SELECT MAX(population)
                          FROM facts);

Done.


name
World


We should recalculate the summary statistics excluding the row for the whole world.

In [13]:
%%sql 
SELECT MIN(population) AS 'Minimum population', 
       MAX(population) AS 'Maximum population',
       MIN(population_growth) AS 'Minimum growth',
       MAX(population_growth) AS 'Maximum growth'
    FROM facts
  WHERE population != (SELECT MAX(population)
                          FROM facts);

Done.


Minimum population,Maximum population,Minimum growth,Maximum growth
0,1367485388,0.0,4.02


The average value for the population and area.

In [15]:
%%sql 
SELECT AVG(population) AS 'Average population', AVG(area) AS 'Average area'
    FROM facts;

Done.


Average population,Average area
62094928.32231405,555093.546184739


## Find all countries meeting criteria

We need to find all countries meeting both of the follwoing criteria : 

- The population is above average.
- The area is below average.

1. Show countries name, population and area.
2. There isn't country name 'world'.
3. countries should meet both criteria.

In [18]:
%%sql 
SELECT name, population, area 
    FROM facts 
  WHERE population != (SELECT MAX(population)
                          FROM facts) AND 
        population > (SELECT AVG(population) 
                          FROM facts) AND 
        area > (SELECT AVG(area) 
                    FROM facts);

Done.


name,population,area
Brazil,204259812,8515770
China,1367485388,9596960
"Congo, Democratic Republic of the",79375136,2344858
Egypt,88487396,1001450
Ethiopia,99465819,1104300
France,66553766,643801
India,1251695584,3287263
Indonesia,255993674,1904569
Iran,81824270,1648195
Mexico,121736809,1964375


### Q1. Which country has the most people? 

In [20]:
%%sql 
SELECT name, population, area 
    FROM facts 
  WHERE population != (SELECT MAX(population)
                          FROM facts) AND 
        population > (SELECT AVG(population) 
                          FROM facts) AND 
        area > (SELECT AVG(area) 
                    FROM facts)
  ORDER BY population DESC
  LIMIT 1;

Done.


name,population,area
China,1367485388,9596960


### Q2. Which country has the highest growth rate? 

In [26]:
%%sql 
SELECT name, population_growth 
    FROM facts 
  WHERE population != (SELECT MAX(population)
                          FROM facts)
  ORDER BY population_growth DESC
  LIMIT 1;

Done.


name,population_growth
South Sudan,4.02


### Q3. Which countries have the highest ratios of water to land?

In [39]:
%%sql
SELECT name, 
       CAST(area_water AS Float) / CAST(area_land AS Float) AS ratio_wl
    FROM facts
  ORDER BY ratio_wl DESC
  LIMIT 1; 

Done.


name,ratio_wl
British Indian Ocean Territory,905.6666666666666


### Q4. Which countries have more water than land?

In [40]:
%%sql 
SELECT name, area_water, area_land 
    FROM facts
  WHERE area_water > area_land;

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


### Q5, Which countries will add the most people to their populations next year?

In [42]:
%%sql 
SELECT name, birth_rate 
    FROM facts
  ORDER BY birth_rate DESC
  LIMIT 10;

Done.


name,birth_rate
Niger,45.45
Mali,44.99
Uganda,43.79
Zambia,42.13
Burkina Faso,42.03
Burundi,42.01
Malawi,41.56
Somalia,40.45
Angola,38.78
Mozambique,38.58


### Q6. Which countries have a higher death rate than birth rate? 

In [44]:
%%sql 
SELECT name, birth_rate, death_rate
    FROM facts
  WHERE birth_rate < death_rate
  ORDER BY name
  LIMIT 10;

Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


### Q7. Which countries have the highest population/area ratio?

In [46]:
%%sql 
SELECT name, (CAST(population AS Float) / area_land) AS population_density
    FROM facts
  ORDER BY population_density DESC
  LIMIT 1;

Done.


name,population_density
Macau,21168.964285714286
